In [1]:
def GrGr(a_1, Mp, Zgrfactor):
    Reset_initial = True

    if Reset_initial:
        %run -i 'initial.py'

    import numpy as np
    import matplotlib.pyplot as plt
    import h5py


    from dedalus import public as de
    from dedalus.extras.plot_tools import quad_mesh, pad_limits

    import logging
    logger = logging.getLogger(__name__)

    %matplotlib inline

    import time

    # from IPython.core.display import display, HTML
    # display(HTML("<style>.container { width:100% !important; }</style>"))

    from dedalus.extras import plot_tools

    a_1 = a_1 # Location of disk 
    AU = 1.496e13
    Mp = Mp # Earth masses
    T_disk = 300 *a_1**(-1/2)

    rho_disk = 2.4e-9*a_1**(-11/4)
    r_Hill = 2e11*a_1*(Mp)**(1/3)
    r_Bondi = 4e10*a_1**(1/2)*Mp

    # Basis and domain

    resolution = 256 #500
    # normalization factors
    r_0 = 1.2e8
    T_0 = 1000
    P_0 = 1 #1e5 #1
    M_0 = 1e-12
    # r_outer = 1.496e13/r_0
    r_outer = 1.2e12/r_0
    r_outer = r_Hill / r_0 #3.4199512e11/r_0
    r_inner = 1.2e9/r_0

    log_inner = np.log(r_inner) 
    log_outer = np.log(r_outer)

    log_basis = de.Chebyshev('r', resolution, interval=(log_inner,log_outer))  
    domain = de.Domain([log_basis], np.float64)

    # Problem

    ncc_cutoff = 1e-10 #1e-2
    tolerance = 1e-10 #1e-2
    # P and T are actually log(P) and log(T)
    problem = de.NLBVP(domain,variables = ['lgP', 'lgT', 'lgM', 'lgZgr'], ncc_cutoff = ncc_cutoff)

    # Parameters

    problem.parameters['T_0'] = T_0 
    problem.parameters['r_0'] = r_0
    problem.parameters['P_0'] = P_0 
    problem.parameters['M_0'] = M_0

    lgP = domain.new_field(name='lgP')
    lgT = domain.new_field(name='lgT')
    lgZgr = domain.new_field(name='lgZgr')
    lgM = domain.new_field(name='lgM')

    r = domain.new_field(name='R')
    r['g'] = domain.grid(0)
    S = np.exp(r['g']) * r_0

    pi = np.pi 
    Core_mass = Mp*5.972e27
    Mc = Core_mass # 5* mass of earth (in g)
    mu = 2.34 * 1.6735575e-24 #mH multiplied by hydrogen atom mass 
    kb = 1.38064852e-16 # g*cm**2/(K*s**2)
    G = 6.67408e-8 # cm**3/(g*s**2)
    dMtot = 1e-5*5.972e27/3.154e7 # 10e-5* mass of earth, g/s #3.154e7 is conversion from yr to s
    sig = 5.670367e-5 # (cgs units) e-5 in g/(K^4s^3), Stefan Boltzmann Const (normally it's e-8 in [W⋅m−2⋅K−4])
    s0 = 1e-4 # cm
    rho_o = 3 #g cm^-3 grain internal density
    sigma_b = 5.6704e-5 # erg*cm^-2*s^-1*K^-4 Stefan Boltzmann 
    rcore = 1.2e9
    grad_rad_cst = - 3*dMtot/(64*np.pi*rcore*sig)

    rhodisk = rho_disk # 7.5e-9 #g/cm**3
    Tdisk = T_disk # 370 # kelvin

    Core_mass = Mp*5.972e27 # 5*5.972e27
    problem.parameters['pi'] = np.pi
    problem.parameters['Mc'] = Core_mass # 5* mass of earth (in g)
    problem.parameters['mu'] = mu #mH multiplied by hydrogen atom mass 
    problem.parameters['kb'] = kb # g*cm**2/(K*s**2)
    problem.parameters['G'] = G # cm**3/(g*s**2)
    problem.parameters['dMtot'] = dMtot # 10e-5* mass of earth, g/s #3.154e7 is conversion from yr to s
    problem.parameters['sig'] = sig # (cgs units) e-5 in g/(K^4s^3), Stefan Boltzmann Const (normally it's e-8 in [W⋅m−2⋅K−4])
    problem.parameters['s0'] = s0 # cm
    problem.parameters['rho_o'] = rho_o #g cm^-3 grain internal density
    problem.parameters['sigma_b'] = sigma_b # erg*cm^-2*s^-1*K^-4 Stefan Boltzmann 
    problem.parameters['grad_rad_cst'] = grad_rad_cst

    problem.parameters['rhodisk'] = rhodisk #g/cm**3
    problem.parameters['Tdisk'] = Tdisk # kelvin

    problem.substitutions['s'] = "(exp(lgM)*M_0*3/(4*pi*rho_o))**(1/3)" # radius
    s = (np.exp(lgM)*M_0*3/(4*pi*rho_o))**(1/3)

    # Opacity


    problem.substitutions['x'] = '2*pi*s*0.3/0.2898'
    x = 2*pi*s*0.3/0.2898

    k0 = -10. # -30.
    problem.parameters['k0'] = k0

    problem.substitutions['Qe'] = "(0.5*(2**k0 + (x*exp(lgT)*T_0)**k0))**(1/k0)"
    Qe = (0.5*(2**k0 + (x*np.exp(lgT)*T_0)**k0))**(1/k0)
    # problem.substitutions['Qe'] = "x*exp(lgT)*T_0 + 0.5*(2 - x*exp(lgT)*T_0)*tanh((x*T_0*exp(lgT) - 2)/2*10 + 1)" 

    problem.substitutions['Qk_geo'] = "Qe*3/4*exp(lgZgr)/(s*rho_o)"
    Qk_geo = Qe*3/4*np.exp(lgZgr)/(s*rho_o)

    k_gas_cst = 1e-8*(problem.parameters["mu"]/problem.parameters['kb'])**(2/3)
    problem.parameters['k_gas_cst'] = k_gas_cst

    problem.substitutions['k_gas'] = "k_gas_cst*(exp(lgT)*T_0)**(7/3)*(exp(lgP)*P_0)**(2/3)"
    k_gas = k_gas_cst*(np.exp(lgT)*T_0)**(7/3)*(np.exp(lgP)*P_0)**(2/3)

    problem.substitutions['kappa'] = "k_gas + Qk_geo"
    kappa = k_gas + Qk_geo 

    # Gradient  

    # problem.parameters['grad_rad_cst'] = -3/64*np.pi*problem.parameters['dMtot']/(problem.parameters['sigma_b']*rcore)
    problem.substitutions['grad_rad'] = "grad_rad_cst*kappa*exp(lgP)*P_0/(exp(lgT)*T_0)**4"
    grad_rad = grad_rad_cst*kappa*np.exp(lgP)*P_0/(np.exp(lgT)*T_0)**4

    grad_ad = 0.28
    problem.parameters['grad_ad'] = grad_ad

    # min
    k1 = -10. # -30.
    problem.parameters['k1'] = k1

    grad = (0.5*(grad_rad**k1 + (grad_ad)**k1))**(1/k1)
    problem.substitutions['grad'] = "(0.5*(grad_rad**k1 + (grad_ad)**k1))**(1/k1)" 

    # Tstop: v_set = gm/r^2 * Tstop 

    gm = problem.parameters['Mc']*problem.parameters['G']
    problem.parameters['gm'] = gm

    problem.substitutions['rho_g'] = '(exp(lgP)*P_0*mu)/(kb*exp(lgT)*T_0)'
    rho_g = (np.exp(lgP)*P_0*mu)/(kb*np.exp(lgT)*T_0)

    problem.substitutions['cg'] = 'sqrt(kb*exp(lgT)*T_0/mu)'
    cg = np.sqrt(kb*np.exp(lgT)*T_0/mu)

    # problem.substitutions['Lg'] = '1.30537485e-9/rho_g'
    problem.substitutions['Lg'] = '1e-9/rho_g'
    Lg = 1e-9/rho_g

    l = 10 #1 #15
    problem.parameters['l'] = l
    problem.substitutions['max'] = "(0.5*(1 + (s/Lg)**l))**(1/l)"
    Max = (0.5*(1 + (s/Lg)**l))**(1/l)

    problem.substitutions['Tstop'] = '(max*4*rho_o*s)/(9*cg*rho_g)'
    Tstop = (Max*4*rho_o*s)/(9*cg*rho_g)

    problem.substitutions['v_set'] = 'gm*Tstop/(exp(r)*r_0)**2'
    v_set = gm*Tstop/(np.exp(r)*r_0)**2

    # Tgrow 

    problem.substitutions['v_bm'] = 'sqrt(16*kb*exp(lgT)*T_0/(pi*exp(lgM)*M_0))'
    v_bm = np.sqrt(16*kb*np.exp(lgT)*T_0/(np.pi*np.exp(lgM)*M_0))

    problem.substitutions['v_dd'] = '0.1*v_set'
    v_dd = 0.1*v_set

    problem.substitutions['Tgrow_inv'] = '3*exp(lgZgr)*rho_g*(v_bm+v_dd)/(rho_o*s)'
    Tgrow_inv = 3*np.exp(lgZgr)*rho_g*(v_bm+v_dd)/(rho_o*s)

    # New equations

    dMdep = 5e-9*5.972e27/3.154e7 
    problem.parameters['dMdep'] = dMdep
    
    problem.parameters['Zgrfactor'] = Zgrfactor 
    
    problem.add_equation('lgZgr = log(dMdep/(4*pi*gm*Tstop*rho_g))')
    problem.add_equation('dr(lgM) = -Tgrow_inv*(exp(r)*r_0)**3/(gm*Tstop)')

    # Normalized equations

    # problem.parameters['eq1cst'] = -1*problem.parameters['G']*problem.parameters['Mc']*problem.parameters['mu']/(T_0*r_0*problem.parameters['kb'])
    problem.add_equation('exp(r) * dr(lgP) = -gm*mu/(T_0*r_0*kb*exp(lgT))')
    problem.add_equation('dr(lgT) = dr(lgP)*grad')

    # Boundary Equations
    

    problem.add_bc("right(lgT) = log(Tdisk/T_0)") # disk temp in kelvins
    problem.add_bc("right(lgP) = log(rhodisk*kb*Tdisk/mu/P_0)") # gas law
    problem.add_bc("right(lgZgr) = log(1e-3*Zgrfactor)")

    # solver = problem.build_solver(de.timesteppers.RK443)
    solver = problem.build_solver()

    # initial conditions and referencing local grid state fields

    lgr = domain.grid(0)
    R = np.exp(lgr)*r_0
    lgT1 = solver.state['lgT']
    lgP1 = solver.state['lgP']
    lgZgr1 = solver.state['lgZgr']
    lgM1 = solver.state['lgM']

    # Load from constant version
    lgP_initial = np.loadtxt("Pressure.txt")
    lgT_initial = np.loadtxt("Temperature.txt")
    lgP1['g'] = lgP_initial
    lgT1['g'] = lgT_initial
    ones = lgT1['g']/lgT1['g']


    if not Reset_initial:
        lgM1['g'] = np.loadtxt('M.txt')
        lgZgr1['g'] = np.loadtxt('Zgr.txt')

    else:
        lgZgr1['g'] = np.log(ones * 1e-2)
        #previously s was 1e-4
        lgM1['g'] = np.log(ones * 4/3*np.pi*problem.parameters['rho_o']*(1e-4)**3/ M_0) 

    lgT_list = [np.copy(lgT['g'])]
    lgP_list = [np.copy(lgP['g'])]
    lgZgr_list = [np.copy(lgZgr['g'])]
    lgM_list = [np.copy(lgM['g'])]

    # Iterations

    i = 0

    pert = solver.perturbations.data
    pert.fill(1+tolerance)
    start_time = time.time()
    while np.sum(np.abs(pert)) > tolerance:
        solver.newton_iteration()
        lgT_list.append(np.copy(lgT1['g'])) # save
        lgP_list.append(np.copy(lgP1['g']))
        lgZgr_list.append(np.copy(lgZgr1['g'])) 
        lgM_list.append(np.copy(lgM1['g'])) 
        logger.info('Perturbation norm: {}'.format(np.sum(np.abs(pert))))
        logger.info('lgT iterate: {}'.format(lgT1['g'][0]))
        logger.info('lgP iterate: {}'.format(lgP1['g'][0]))
        logger.info('lgM iterate: {}'.format(lgM1['g'][0]))
        logger.info('lgZgr iterate: {}'.format(lgZgr1['g'][0]))

    end_time = time.time()

    R = np.log10(np.exp(domain.grid(0))*r_0)

    np.savetxt("Pressure.txt", lgP_list[-1])
    np.savetxt("Temperature.txt", lgT_list[-1])
    np.savetxt("Zgr.txt", lgZgr_list[-1])
    np.savetxt("M.txt", lgM_list[-1])





In [2]:
    
def EnEq(a_1, Mp, Zgrfactor):
    
    GrGr(a_1, Mp, Zgrfactor)
    
    load_dep = True

    import numpy as np
    import os 
    from dedalus import public as de
    from dedalus.extras.plot_tools import quad_mesh, pad_limits
    import matplotlib.pyplot as plt
    from scipy.special import erf 
    import logging
    logger = logging.getLogger(__name__)

    %matplotlib inline

    import time

    # from IPython.core.display import display, HTML
    # display(HTML("<style>.container { width:100% !important; }</style>"))

    from dedalus.extras import plot_tools 

    a_1 = a_1
    AU = 1.496e13
    Mp = Mp
    T_disk = 300 *a_1**(-1/2)

    rho_disk = 2.4e-9*a_1**(-11/4)
    r_Hill = 2e11*a_1*(Mp)**(1/3)
    r_Bondi = 4e10*a_1**(1/2)*Mp

    # Basis and domain

    resolution = 256 #500
    # normalization factors
    r_0 = 1.2e8
    T_0 = 1000
    P_0 = 1 #1e5 #1
    M_0 = 1e-12
    m_0 = 1e25
    L_0 = 3.144546892817586e+27
    Dep_0 = 5.972e27/3.154e7 

    # r_outer = 1.496e13/r_0
    r_outer = r_Hill /r_0 # 3.4199512e11/r_0
    r_inner = 1.2e9/r_0

    log_inner = np.log(r_inner) 
    log_outer = np.log(r_outer)

    log_basis = de.Chebyshev('r', resolution, interval=(log_inner,log_outer))  
    domain = de.Domain([log_basis], np.float64)

    # Problem

    ncc_cutoff = 1e-6 #1e-2
    tolerance = 1e-6 #1e-2
    # P and T are actually log(P) and log(T)
    problem = de.NLBVP(domain,variables = ['lgP', 'lgT', 'lgM', 'lgZgr','m', 'L'], ncc_cutoff = ncc_cutoff)
    R = np.log10(np.exp(domain.grid(0))*r_0)


    # Parameters

    problem.parameters['T_0'] = T_0 
    problem.parameters['r_0'] = r_0
    problem.parameters['P_0'] = P_0 # unused
    problem.parameters['M_0'] = M_0
    problem.parameters['m_0'] = m_0
    problem.parameters['Dep_0'] = Dep_0
    problem.parameters['L_0'] = L_0

    lgP = domain.new_field(name='lgP')
    lgT = domain.new_field(name='lgT')
    lgZgr = domain.new_field(name='lgZgr')
    lgM = domain.new_field(name='lgM')
    m = domain.new_field(name='m')
    L = domain.new_field(name='L')

    r = domain.new_field(name='R')
    r['g'] = domain.grid(0)
    S = np.exp(r['g']) * r_0
    ones = np.ones(len(r['g']))

    pi = np.pi 
    Core_mass = Mp*5.972e27
    Mc = Core_mass # 5* mass of earth (in g)
    mu = 2.34 * 1.6735575e-24 #mH multiplied by hydrogen atom mass 
    kb = 1.38064852e-16 # g*cm**2/(K*s**2)
    G = 6.67408e-8 # cm**3/(g*s**2)
    dMtot = 0.2*1e-5* 5.972e27/3.154e7 # 10e-5* mass of earth, g/s #3.154e7 is conversion from yr to s
    sig = 5.670367e-5 # (cgs units) e-5 in g/(K^4s^3), Stefan Boltzmann Const (normally it's e-8 in [W⋅m−2⋅K−4])
    s0 = 1e-4 # cm
    rho_o = 3 #g cm^-3 grain internal density
    sigma_b = 5.6704e-5 # erg*cm^-2*s^-1*K^-4 Stefan Boltzmann 
    rcore = 1.2e9
    grad_rad_cst = - 3*dMtot/(64*np.pi*rcore*sig)
    gm = G*Mc 

    rhodisk = rho_disk # 7.5e-9 #g/cm**3
    Tdisk = T_disk # 370 # kelvin

    mdep = rho_o * 4/3 * pi *(1e4)**3
    dMdisk = 5e-9 * 5.972e27/3.154e7 


    problem.parameters['pi'] = np.pi
    problem.parameters['Mc'] = Core_mass 
    problem.parameters['mu'] = 2.34 * 1.6735575e-24 #mH multiplied by hydrogen atom mass 
    problem.parameters['kb'] = 1.38064852e-16 # g*cm**2/(K*s**2)
    problem.parameters['G'] = 6.67408e-8 # cm**3/(g*s**2)
    problem.parameters['dMtot'] = 0.2*1e-5*5.972e27/3.154e7 # 10e-5* mass of earth, g/s #3.154e7 is conversion from yr to s
    problem.parameters['sig'] = 5.670367e-5 # (cgs units) e-5 in g/(K^4s^3), Stefan Boltzmann Const (normally it's e-8 in [W⋅m−2⋅K−4])
    problem.parameters['s0'] = 1e-4 # cm
    problem.parameters['rho_o'] = 3 #g cm^-3 grain internal density
    problem.parameters['mdep'] = problem.parameters['rho_o']*4/3*np.pi*(1e-4)**(3)
    problem.parameters['sigma_b'] = 5.6704e-5 # erg*cm^-2*s^-1*K^-4 Stefan Boltzmann 
    rcore = 1.2e9
    problem.parameters['grad_rad_cst'] = - 3*problem.parameters['dMtot']/(64*np.pi*rcore*problem.parameters['sig'])


    # problem.parameters['rhodisk'] = 4e-9 # 1e-11 #g/cm**3
    # problem.parameters['Tdisk'] = 370 # 150 # kelvin

    problem.parameters['rhodisk'] = rhodisk #g/cm**3
    problem.parameters['Tdisk'] = Tdisk # kelvin

    problem.substitutions['s'] = '(exp(lgM)*M_0*3/(4*pi*rho_o))**(1/3)' # radius
    s = (np.exp(lgM)*M_0*3/(4*pi*rho_o))**(1/3)

    # # Mass deposition 
    deposition_version = 'null'

    if deposition_version == 0:
        lgdM = domain.new_field(name='lgdM')
        lgdM['g'] = np.loadtxt('M1dep.txt')

        d_r = log_basis.Differentiate(lgdM)
        dlgdM = d_r.evaluate()
        Mdep_fixed = lgdM['g']
        dlgdM_fixed = dlgdM['g']
        n_roll = Mdep_fixed.shape[0]//4
        Mdep_fixed_rolled =np.roll(Mdep_fixed, n_roll)
        dlgdM_fixed_rolled = np.roll(dlgdM_fixed, n_roll)

        # Correct leftmost values too low from wrapping around
        Mdep_fixed_rolled[:n_roll] = Mdep_fixed[0]
        dlgdM_fixed_rolled[:n_roll] = 0. # Zero gradient for constant value correction

        ones = lgdM['g']/lgdM['g']

        constant = np.log(ones*1e-5)
        dconstant = 0.*ones

        lgdM['g'] = Mdep_fixed_rolled
        dlgdM['g'] = dlgdM_fixed_rolled



    elif deposition_version == 1:
        lgdM = domain.new_field(name='lgdM')
        lgdM['g'] = np.log(np.loadtxt('Deposition.txt')/Dep_0)
        d_r = log_basis.Differentiate(lgdM)
        dlgdM = d_r.evaluate()

    # elif deposition_version=='None':
    #     lgdM = domain.new_field(name='lgdM')
    #     lgdM['g'] = np.log(np.loadtxt('Deposition.txt')/Dep_0)
    #     d_r = log_basis.Differentiate(lgdM)
    #     dlgdM = d_r.evaluate()

    elif deposition_version=='null':
        location = 10 # in log10 
        smooth = 3
        X = r['g'] - np.log(10**location/r_0) 
        lgdM = domain.new_field(name='lgdM')
        lgdM['g'] = X*0
        lgdM['g'] = lgdM['g']+np.log(dMdisk/Dep_0)
        print(np.log(dMdisk/Dep_0))
        # DEP = dMdisk + (dMtot-dMdisk)*((erf(smooth*eX['g']) + 1)/2)
        d_r = log_basis.Differentiate(lgdM)
        dlgdM = d_r.evaluate()

    else:
        location = 10 # in log10 
        smooth = 3
        X = r['g'] - np.log(10**location/r_0) 
        lgdM = domain.new_field(name='lgdM')
        lgdM['g'] = np.log((dMdisk + (dMtot-dMdisk)*((-erf(smooth*X) + 1)/2))/Dep_0)    
        # DEP = dMdisk + (dMtot-dMdisk)*((erf(smooth*eX['g']) + 1)/2)
        d_r = log_basis.Differentiate(lgdM)
        dlgdM = d_r.evaluate()

    problem.parameters['lgDep'] = lgdM
    problem.parameters['dlgDep'] = dlgdM

    lgDep = domain.new_field(name='lgDep')
    dlgDep = domain.new_field(name='dlgDep')
    lgDep['g'] = lgdM['g']
    dlgDep['g'] = dlgdM['g']

    # Opacity

    problem.substitutions['x'] = '2*pi*s*0.3/0.2898'
    x = 2*pi*s*0.3/0.2898

    # min(2,ex*T*T_0) 
    k0 = -10. # -30.
    problem.parameters['k0'] = k0

    problem.substitutions['Qe'] = "(0.5*(2**k0 + (x*exp(lgT)*T_0)**k0))**(1/k0)"
    Qe = (0.5*(2**k0 + (x*np.exp(lgT)*T_0)**k0))**(1/k0)

    problem.substitutions['Qk_geo'] = "Qe*3/4*exp(lgZgr)/(s*rho_o)" 
    Qk_geo = Qe*3/4*np.exp(lgZgr)/(s*rho_o)

    k_gas_cst = 1e-8*(problem.parameters["mu"]/problem.parameters['kb'])**(2/3) # O(10^-14)
    problem.parameters['k_gas_cst'] = k_gas_cst

    problem.substitutions['k_gas'] = "k_gas_cst*(exp(lgT)*T_0)**(7/3)*(exp(lgP)*P_0)**(2/3)" 
    k_gas = k_gas_cst*(np.exp(lgT)*T_0)**(7/3)*(np.exp(lgP)*P_0)**(2/3)

    problem.substitutions['kappa'] = "k_gas + Qk_geo"
    kappa = k_gas + Qk_geo 

    # Gradient  


    # problem.substitutions['grad_rad'] = "grad_rad_cst*kappa*exp(lgP)*P_0/(exp(lgT)*T_0)**4"

    grad_ad = 0.28
    problem.parameters['grad_ad'] = grad_ad


    # problem.substitutions['grad_rad'] = '-3*kappa*exp(L)*L_0/(64*pi*sigma_b*gm) * exp(lgP)*P_0/(exp(lgT)*T_0)**4'
    problem.substitutions['grad_rad'] = '3*kappa*L*L_0/(64*pi*sigma_b*gm) * exp(lgP)*P_0/(exp(lgT)*T_0)**4'
    grad_rad = 3*kappa*L*L_0/(64*pi*sigma_b*gm) * np.exp(lgP)*P_0/(np.exp(lgT)*T_0)**4


    # min
    k1 = -10. # -30.
    problem.parameters['k1'] = k1
    problem.substitutions['grad'] = "(0.5*(grad_rad**k1 + (grad_ad)**k1))**(1/k1)" 
    grad = (0.5*(grad_rad**k1 + (grad_ad)**k1))**(1/k1)


    # Tstop: v_set = gm/r^2 * Tstop 

    gm = problem.parameters['Mc']*problem.parameters['G']
    problem.parameters['gm'] = gm

    problem.substitutions['rho_g'] = '(exp(lgP)*P_0*mu)/(kb*exp(lgT)*T_0)'
    rho_g = (np.exp(lgP)*P_0*mu)/(kb*np.exp(lgT)*T_0)

    problem.substitutions['cg'] = 'sqrt(kb*exp(lgT)*T_0/mu)'
    cg = np.sqrt(kb*np.exp(lgT)*T_0/mu)

    problem.substitutions['Lg'] = '1e-9/rho_g'
    Lg = 1e-9/rho_g

    l = 30 #1 #15
    problem.parameters['l'] = l

    problem.substitutions['max'] = "(0.5*(1 + (s/Lg)**l))**(1/l)"
    Max = (0.5*(1 + (s/Lg)**l))**(1/l)


    problem.substitutions['Tstop'] = '(max*4*rho_o*s)/(9*cg*rho_g)'
    Tstop = (Max*4*rho_o*s)/(9*cg*rho_g)

    problem.substitutions['v_set'] = 'gm*Tstop/(exp(r)*r_0)**2'
    v_set = gm*Tstop/(np.exp(r)*r_0)**2

    # Tgrow 

    problem.substitutions['v_bm'] = 'sqrt(16*kb*exp(lgT)*T_0/(pi*exp(lgM)*M_0))'
    v_bm = np.sqrt(16*kb*np.exp(lgT)*T_0/(np.pi*np.exp(lgM)*M_0))

    problem.substitutions['v_dd'] = '0.1*v_set'
    v_dd = 0.1*v_set

    problem.substitutions['Tgrow_inv'] = '3*exp(lgZgr)*rho_g*(v_bm+v_dd)/(rho_o*s)'
    Tgrow_inv = 3*np.exp(lgZgr)*rho_g*(v_bm+v_dd)/(rho_o*s)


    # Energy equation

    # problem.substitutions['drho_dep'] = '-1/(4*pi*(exp(r)*r_0)**2) * dlgDep * 1e5'
    problem.substitutions['drho_dep'] = '-1/(4*pi*(exp(r)*r_0)**3) * dlgDep * Dep_0 * exp(lgDep)'
    drho_dep = -1/(4*pi*(np.exp(r)*r_0)**3) * dlgDep * Dep_0 * np.exp(lgDep)

    Trelax = 1e30
    problem.parameters['Trelax'] = Trelax

    Lcore = problem.parameters['gm']*problem.parameters['dMtot']/(r_inner*r_0)
    problem.parameters['Lcore'] = Lcore
    
    problem.parameters['Zgrfactor'] = Zgrfactor
    
    
    # New equations

    problem.add_equation('lgZgr = log(exp(lgDep)*Dep_0/(4*pi*gm*Tstop*rho_g))')
    # problem.add_equation('lgZgr = log(dMdep/(4*pi*gm*Tstop*rho_g))')
    problem.add_equation('dr(lgM) = -Tgrow_inv*(exp(r)*r_0)**3/(gm*Tstop) + dlgDep*(mdep-exp(lgM)*M_0)/(exp(lgM)*M_0)')


    # Normalized equations

    # problem.parameters['eq1cst'] = -1*problem.parameters['G']*problem.parameters['Mc']*problem.parameters['mu']/(T_0*r_0*problem.parameters['kb'])
    problem.add_equation('dr(m) = 4*pi*(exp(r)*r_0)**3 * rho_g/m_0')
    problem.add_equation('exp(r) * dr(lgP) = -G*(Mc+m*m_0)*mu/(T_0*r_0*kb*exp(lgT))')
    problem.add_equation('dr(lgT) = dr(lgP)*grad')

    # problem.add_equation('dr(L) = 4*pi*(exp(r)*r_0)* (G*(Mc+m*m_0)*drho_dep/(exp(r)*r_0))')
    # problem.add_equation('dr(L)*L_0 = 4*pi*(exp(r)*r_0)**3 * (G*(Mc+m*m_0)*drho_dep/(exp(r)*r_0) + rho_g*cg*(exp(lgT)*T_0 - Tdisk)/Trelax )')
    problem.add_equation('dr(L)*L_0 = (4*pi*(exp(r)*r_0)**3 * (G*(Mc+m*m_0)*drho_dep/(exp(r)*r_0) + rho_g*cg*(exp(lgT)*T_0 - Tdisk)/Trelax ))')


    # Boundary Equations

     
    
    problem.add_bc("right(lgT) = log(Tdisk/T_0)") # disk temp in kelvins
    problem.add_bc("right(lgP) = log(rhodisk*kb*Tdisk/mu/P_0)") # gas law
    problem.add_bc("right(lgZgr) = log(1e-3*Zgrfactor)")
    problem.add_bc("left(m) = 0")
    problem.add_bc("left(L) = Lcore/L_0")

    # solver = problem.build_solver(de.timesteppers.RK443)
    solver = problem.build_solver()


    # initial conditions and referencing local grid state fields

    lgr = domain.grid(0)
    Rscale = np.log10(np.exp(domain.grid(0))*r_0)
    lgT1 = solver.state['lgT']
    lgP1 = solver.state['lgP']
    lgZgr1 = solver.state['lgZgr']
    lgM1 = solver.state['lgM']

    L1 = solver.state['L']
    m1= solver.state['m']

    # Load from constant Mass and Zgr version

    lgP_initial = np.loadtxt("Pressure.txt")
    lgT_initial = np.loadtxt("Temperature.txt")
    Zi = np.loadtxt('Zgr.txt')
    Mi = np.loadtxt('M.txt')

    lgP1['g'] = lgP_initial
    lgT1['g'] = lgT_initial
    ones = lgT1['g']/lgT1['g']

    L1['g'] = ones*problem.parameters['Lcore']/L_0

    lgZgr1['g'] = Zi 

    lgM1['g'] = Mi
    m1['g'] = ones*0


    if load_dep: 
        lgP1['g'] = np.loadtxt("Pressure_dep.txt")
        lgT1['g'] = np.loadtxt("Temperature_dep.txt")
        L1['g'] = np.loadtxt('L_dep.txt')
        lgZgr1['g'] = np.loadtxt('Zgr_dep.txt') 
        lgM1['g'] = np.loadtxt('M_dep.txt')
        m1['g'] = np.loadtxt('m_dep.txt')



    lgT_list = [np.copy(lgT1['g'])]
    lgP_list = [np.copy(lgP1['g'])]
    lgZgr_list = [np.copy(lgZgr1['g'])]
    lgM_list = [np.copy(lgM1['g'])]
    m_list = [np.copy(m1['g'])]
    L_list = [np.copy(L1['g'])]


    # Iterations

    i = 0

    pert = solver.perturbations.data
    pert.fill(1+tolerance)
    start_time = time.time()
    while np.sum(np.abs(pert)) > tolerance:
        solver.newton_iteration()
        lgT_list.append(np.copy(lgT1['g'])) # save
        lgP_list.append(np.copy(lgP1['g']))
        lgZgr_list.append(np.copy(lgZgr1['g'])) 
        lgM_list.append(np.copy(lgM1['g'])) 
        L_list.append(np.copy(L1['g']))
        m_list.append(np.copy(m['g']))
        logger.info('Perturbation norm: {}'.format(np.sum(np.abs(pert))))
        logger.info('lgT iterate: {}'.format(lgT1['g'][0]))
        logger.info('lgP iterate: {}'.format(lgP1['g'][0]))
        logger.info('lgM iterate: {}'.format(lgM1['g'][0]))
        logger.info('lgZgr iterate: {}'.format(lgZgr1['g'][0]))
        logger.info('m iterate: {}'.format(m1['g'][0]))
        logger.info('L iterate: {}'.format(L1['g'][0]))


    end_time = time.time()




    R = np.log10(np.exp(domain.grid(0))*r_0)

    lgP['g'] = lgP_list[-1]
    lgT['g'] = lgT_list[-1]
    lgZgr['g'] = lgZgr_list[-1]
    lgM['g'] = lgM_list[-1]
    m['g'] = m_list[-1]
    L['g'] = L_list[-1]

    # Grad

    Grad = grad.evaluate()
    Grad.require_grid_space()
    Grad_rad = grad_rad.evaluate()
    Grad_rad.require_grid_space()

    RCB = 0
    epsilon = 0.0001
    for i in range(0, len(Grad['g'] - 1)):
        if np.abs(Grad['g'][i] - Grad['g'][0]) >  epsilon:
            RCB = i
            break

    TStop = Tstop.evaluate()
    TStop.require_grid_space()

    Tgrow = 1/Tgrow_inv
    TGrow = Tgrow.evaluate()
    TGrow.require_grid_space()

    Bondi = r_Bondi # cm
    # Bondi = 3.7e11 # cm
    # Tsettl = r/vsettl 
    def find_nearest_index(A, x):
        i = (np.abs(A - x)).argmin()
        return i

    bondi = find_nearest_index(R, np.log10(Bondi))
    Vsettl = v_set.evaluate()
    Vsettl.require_grid_space()
    Vset = Vsettl['g']

    Tsettl = np.ones(len(R))

    for i in range(0, len(R)):
        Tsettl[i] = np.trapz(1/Vset[0:i], S[0:i]) # Ormel integral definition 

    Trcb_set = Tsettl[RCB]
    Tbondi_set = Tsettl[bondi]
    Tout_set = Tsettl[-1]

    yr = 3.154e+7

    Bondi = 3.7e11 # cm
    Bondi = 8.9442713e10
    # Tsettl = r/vsettl 
    def find_nearest_index(A, x):
        i = (np.abs(A - x)).argmin()
        return i

    bondi = find_nearest_index(R, np.log10(Bondi))
    Vsettl = v_set.evaluate()
    Vsettl.require_grid_space()
    Vset = Vsettl['g']


    Tsettl = np.ones(len(R))


    for i in range(0, len(R)):
        Tsettl[i] = np.trapz(1/Vset[0:i], S[0:i]) # Ormel integral definition 

    # Tsettl = np.exp(r['g'])*r_0/Vset 

    Trcb_set = Tsettl[RCB]
    Tbondi_set = Tsettl[bondi]
    Tout_set = Tsettl[-1]

    yr = 3.154e+7


    size = s.evaluate()
    size.require_grid_space()
    kappa_gr = Qk_geo.evaluate()
    kappa_gr.require_grid_space()


    rho_gas = rho_g.evaluate()
    rho_gas.require_grid_space()

    lgt = lgT_list[-1]


    Env = np.zeros(len(R))

    En = 4*np.pi*(np.exp(r)*r_0)**2 *rho_g 

    m = En.evaluate()
    m.require_grid_space()

    for i in range(0, len(R)):
        Env[i] = np.trapz(m['g'][0:i], S[0:i]) 


    env_mass = Env[-1]

    dlgZgr = lgZgr.differentiate('r')
    dlgP = lgP.differentiate('r')

    # Multiply dlgZgr by Zgr to get dZgr (as a proxy for dmu = d(1+Zgr) = dZgr)
    grad_mu= dlgZgr['g']/ dlgP['g'] 
    grad_mu *= np.exp(lgZgr['g'])

    lgMu = np.log(1+np.exp(lgZgr))
    dlgMu = log_basis.Differentiate(lgMu)
    dlgP = log_basis.Differentiate(np.log(np.exp(lgP) *P_0))

    grad_mu = dlgMu/dlgP

    # grad_mu1 = np.diff( np.log(1+np.exp(lgZgr['g'])))/ np.diff(np.log(np.exp(lgP['g']) *P_0)) 
    Grad_mu = grad_mu.evaluate()
    Grad_mu.require_grid_space()

    grad_diff = grad_rad - grad_mu - (-grad_ad) 
    Grad_diff = grad_diff.evaluate() 
    Grad_diff.require_grid_space() 

    R_0 = (Grad['g'] - grad_ad)/Grad_mu['g']

    c_gas=np.sqrt(8.32e7 * np.exp(lgt)*T_0 )
    cross_sect_H2=2e-15 #cgs H2 cross section
    lambda_mfp= 1./ (cross_sect_H2 * rho_gas['g'] / 2 / 1.67e-24)
    nu_visc= lambda_mfp * c_gas


    gas_gamma=7./5.
    kappa_tot =  kappa.evaluate()
    kappa_tot.require_grid_space()
    #xi_rad= 16./3. * (gas_gamma-1)/gas_gamma * 5.670367e-5 * (np.exp(lgt['g'])*T_0)**4.  / rho_gas['g'] / np.exp(lgP['g']) 
    xi_rad= 16./3. * (gas_gamma-1)/gas_gamma * 5.670367e-5 * (np.exp(lgt)*T_0)**4.  / rho_gas['g'] / kappa_tot['g']/ np.exp(lgP['g'])

    tau_opt= 10.**R * rho_gas['g'] *kappa_tot['g'] # Approximation to true integral of rho.dz

    Cs = cg.evaluate() # sound speed, which we previously used 
    Cs.require_grid_space()
    Cs = Cs['g']

    Cs = cg.evaluate() # sound speed, which we previously used 
    Cs.require_grid_space()
    Cs = Cs['g']

    Nth = np.sqrt((G*(Core_mass+Env)/(10**R)**2)**2/Cs**2*(grad_ad-Grad_rad['g'])) # I changed grad_rad to be positive now
    d_finger= ((nu_visc*xi_rad)/Nth**2)**(1/4)
    tau_cool = d_finger**2/xi_rad

    Rgas = 8.32e7 / 2.3 # You gave me this value, but the wikipedia one in cgs doesnt have the /2.3 
    Sigma = 5.7e-5
    tau_cool_thin = Rgas/(16*kappa_tot['g']*Sigma*(np.exp(lgt)*T_0)**3)

    Prandtl = nu_visc/xi_rad 

    v_coll = d_finger/(np.sqrt(Prandtl)*tau_cool)

    np.savetxt("NPressure_dep.txt", lgP_list[-1])
    np.savetxt("NTemperature_dep.txt", lgT_list[-1])
    np.savetxt("NZgr_dep.txt", lgZgr_list[-1])
    np.savetxt("NM_dep.txt", lgM_list[-1])
    np.savetxt("Nm_dep.txt",  m_list[-1])
    np.savetxt("NL_dep.txt", L_list[-1])

    prefix = "{}_{}_{}_".format(a_1, Mp, Zgrfactor)

    np.savetxt('saved/'+prefix+'Ngrad.txt', Grad['g'])
    np.savetxt('saved/'+prefix+'Ngrad_rad.txt', Grad_rad['g'])
    np.savetxt('saved/'+prefix+'NTstop.txt', np.log10(TStop['g']))
    np.savetxt('saved/'+prefix+'NTgrow.txt', np.log10(TGrow['g']))
    np.savetxt('saved/'+prefix+'NTsettl.txt', Tsettl) 
    np.savetxt('saved/'+prefix+'NZgr.txt', np.exp(lgZgr['g'])) 
    np.savetxt('saved/'+prefix+'Nkappa_gr.txt', kappa_gr['g']) 
    np.savetxt('saved/'+prefix+'Nsize.txt', size['g'])
    np.savetxt('saved/'+prefix+'Nrho_gas_cbrt.txt', (rho_gas['g'])**(1/3))
    np.savetxt('saved/'+prefix+'NTemperature.txt', np.exp(lgt)*T_0)
    np.savetxt('saved/'+prefix+'NPress.txt', np.exp(lgt)*T_0 *rho_gas['g'])
    np.savetxt('saved/'+prefix+'Nrho_gas_norm.txt', (rho_gas['g']/rhodisk)**(1/3))
    np.savetxt('saved/'+prefix+'NTemperature_norm.txt',np.exp(lgt)*T_0 / Tdisk)
    np.savetxt('saved/'+prefix+'NEnv.txt', Env)
    np.savetxt('saved/'+prefix+'Ngrad_mu.txt', Grad_mu['g'])
    np.savetxt('saved/'+prefix+'Ngrad_ad.txt', grad_ad*np.ones(len(R)))
    np.savetxt('saved/'+prefix+'Ngrad_diff.txt',  Grad_diff['g'])
    np.savetxt('saved/'+prefix+'Nlog10(R_0).txt',  np.log10(R_0))
    np.savetxt('saved/'+prefix+'Nlambda_mfp.txt',  np.log10(lambda_mfp))
    np.savetxt('saved/'+prefix+'Nc_gas.txt',  np.log10(c_gas))
    np.savetxt('saved/'+prefix+'Nnu_visc.txt',  np.log10(nu_visc))
    np.savetxt('saved/'+prefix+'Nxi_rad.txt',  np.log10(xi_rad))
    np.savetxt('saved/'+prefix+'Nkappa_tot.txt',  np.log10(kappa_tot['g']))
    np.savetxt('saved/'+prefix+'NTfourth.txt',  np.log10((np.exp(lgt)*T_0)**4.))
    np.savetxt('saved/'+prefix+'Nrho_gas.txt',  np.log10(rho_gas['g']))
    np.savetxt('saved/'+prefix+'NPrandtl_number.txt',  np.log10(nu_visc/xi_rad))
    np.savetxt('saved/'+prefix+'Ntau_opt.txt',  np.log10(tau_opt))
    np.savetxt('saved/'+prefix+'NLedoux.txt',  Grad['g'] - Grad_mu['g'] - grad_ad)
    np.savetxt('saved/'+prefix+'NR.txt', R)


In [3]:
import numpy as np
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

versions = cartesian_product(np.array([0.2,1,5]), np.array([0.2, 1, 5]), np.array([1/10, 1/3, 1, 3]))
print(versions)

[[0.2        0.2        0.1       ]
 [0.2        0.2        0.33333333]
 [0.2        0.2        1.        ]
 [0.2        0.2        3.        ]
 [0.2        1.         0.1       ]
 [0.2        1.         0.33333333]
 [0.2        1.         1.        ]
 [0.2        1.         3.        ]
 [0.2        5.         0.1       ]
 [0.2        5.         0.33333333]
 [0.2        5.         1.        ]
 [0.2        5.         3.        ]
 [1.         0.2        0.1       ]
 [1.         0.2        0.33333333]
 [1.         0.2        1.        ]
 [1.         0.2        3.        ]
 [1.         1.         0.1       ]
 [1.         1.         0.33333333]
 [1.         1.         1.        ]
 [1.         1.         3.        ]
 [1.         5.         0.1       ]
 [1.         5.         0.33333333]
 [1.         5.         1.        ]
 [1.         5.         3.        ]
 [5.         0.2        0.1       ]
 [5.         0.2        0.33333333]
 [5.         0.2        1.        ]
 [5.         0.2        3.  

In [ ]:
# AU, M, Zgr

for A in versions:
    a_1 = A[0]
    Mp = 4 if A[1] == 5 else A[1]
    Zgrfactor = A[2]
    EnEq(a_1, Mp, Zgrfactor)

-471.0568564937796
2020-10-15 05:54:30,770 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 6.5e+01/s
2020-10-15 05:54:30,783 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.5e+02/s
2020-10-15 05:54:30,787 __main__ 0/1 INFO :: Perturbation norm: 17.365219865144315
2020-10-15 05:54:30,788 __main__ 0/1 INFO :: T iterate: 2.268591126035416
2020-10-15 05:54:30,789 __main__ 0/1 INFO :: P iterate: 11.937870498209637
2020-10-15 05:54:30,831 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.1e+02/s
2020-10-15 05:54:30,837 __main__ 0/1 INFO :: Perturbation norm: 4.548874464065178
2020-10-15 05:54:30,837 __main__ 0/1 INFO :: T iterate: 2.4780799537499414
2020-10-15 05:54:30,838 __main__ 0/1 INFO :: P iterate: 12.686044882904369
2020-10-15 05:54:30,855 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.0e+02/s
2020-10-15 05:54:30,861 __mai

2020-10-15 05:55:02,665 __main__ 0/1 INFO :: Perturbation norm: 11.539729865326272
2020-10-15 05:55:02,666 __main__ 0/1 INFO :: lgT iterate: -0.20484280394944399
2020-10-15 05:55:02,667 __main__ 0/1 INFO :: lgP iterate: 11.044306993334606
2020-10-15 05:55:02,668 __main__ 0/1 INFO :: lgM iterate: 14.572620524110567
2020-10-15 05:55:02,668 __main__ 0/1 INFO :: lgZgr iterate: -6.251156283523108
2020-10-15 05:55:05,145 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.7e+00/s
2020-10-15 05:55:05,272 __main__ 0/1 INFO :: Perturbation norm: 11.794283281069719
2020-10-15 05:55:05,273 __main__ 0/1 INFO :: lgT iterate: -0.25400803295020347
2020-10-15 05:55:05,273 __main__ 0/1 INFO :: lgP iterate: 11.068715511741905
2020-10-15 05:55:05,273 __main__ 0/1 INFO :: lgM iterate: 15.71593536939457
2020-10-15 05:55:05,275 __main__ 0/1 INFO :: lgZgr iterate: -7.111522542255292
2020-10-15 05:55:07,510 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s

2020-10-15 05:55:28,077 __main__ 0/1 INFO :: Perturbation norm: 4.548874464065178
2020-10-15 05:55:28,077 __main__ 0/1 INFO :: T iterate: 2.4780799537499414
2020-10-15 05:55:28,078 __main__ 0/1 INFO :: P iterate: 12.686044882904369
2020-10-15 05:55:28,096 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.0e+02/s
2020-10-15 05:55:28,101 __main__ 0/1 INFO :: Perturbation norm: 2.413842169591379
2020-10-15 05:55:28,102 __main__ 0/1 INFO :: T iterate: 2.560627232956435
2020-10-15 05:55:28,102 __main__ 0/1 INFO :: P iterate: 12.98085659435613
2020-10-15 05:55:28,120 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.1e+02/s
2020-10-15 05:55:28,124 __main__ 0/1 INFO :: Perturbation norm: 0.4245041630840209
2020-10-15 05:55:28,125 __main__ 0/1 INFO :: T iterate: 2.588505177251105
2020-10-15 05:55:28,126 __main__ 0/1 INFO :: P iterate: 13.080420681122828
2020-10-15 05:55:28,143 pencil 0/1 INFO :: Building pencil mat

2020-10-15 05:56:05,855 __main__ 0/1 INFO :: lgP iterate: 11.044799345574933
2020-10-15 05:56:05,856 __main__ 0/1 INFO :: lgM iterate: 15.719445960796993
2020-10-15 05:56:05,857 __main__ 0/1 INFO :: lgZgr iterate: -7.066284516044802
2020-10-15 05:56:07,876 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 5.3e+00/s
2020-10-15 05:56:08,011 __main__ 0/1 INFO :: Perturbation norm: 10.913555454263781
2020-10-15 05:56:08,012 __main__ 0/1 INFO :: lgT iterate: -0.2893723687034584
2020-10-15 05:56:08,013 __main__ 0/1 INFO :: lgP iterate: 11.059553525893008
2020-10-15 05:56:08,033 __main__ 0/1 INFO :: lgM iterate: 18.655034645288552
2020-10-15 05:56:08,034 __main__ 0/1 INFO :: lgZgr iterate: -9.11480657724479
2020-10-15 05:56:10,410 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.9e+00/s
2020-10-15 05:56:10,539 __main__ 0/1 INFO :: Perturbation norm: 6.461170890174832
2020-10-15 05:56:10,539 __main__ 0/1 INFO :: lgT

2020-10-15 05:56:27,238 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.5e+02/s
2020-10-15 05:56:27,241 __main__ 0/1 INFO :: Perturbation norm: 0.4245041630840209
2020-10-15 05:56:27,241 __main__ 0/1 INFO :: T iterate: 2.588505177251105
2020-10-15 05:56:27,242 __main__ 0/1 INFO :: P iterate: 13.080420681122828
2020-10-15 05:56:27,253 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.6e+02/s
2020-10-15 05:56:27,256 __main__ 0/1 INFO :: Perturbation norm: 0.010978509098376978
2020-10-15 05:56:27,256 __main__ 0/1 INFO :: T iterate: 2.5896819719802675
2020-10-15 05:56:27,256 __main__ 0/1 INFO :: P iterate: 13.084623519441266
2020-10-15 05:56:27,267 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.6e+02/s
2020-10-15 05:56:27,270 __main__ 0/1 INFO :: Perturbation norm: 5.610693476662668e-06
2020-10-15 05:56:27,271 __main__ 0/1 INFO :: T iterate: 2.5896828587962633
2020-1

...

2020-10-15 06:10:57,538 __main__ 0/1 INFO :: lgP iterate: 8.281043729070499
2020-10-15 06:10:57,539 __main__ 0/1 INFO :: lgM iterate: 20.226611632715866
2020-10-15 06:10:57,540 __main__ 0/1 INFO :: lgZgr iterate: -7.994285958543491
2020-10-15 06:11:00,255 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.6e+00/s
2020-10-15 06:11:00,438 __main__ 0/1 INFO :: Perturbation norm: 2.157183829933473
2020-10-15 06:11:00,439 __main__ 0/1 INFO :: lgT iterate: -0.7965842247427316
2020-10-15 06:11:00,440 __main__ 0/1 INFO :: lgP iterate: 8.30883468979758
2020-10-15 06:11:00,441 __main__ 0/1 INFO :: lgM iterate: 21.28485939935461
2020-10-15 06:11:00,441 __main__ 0/1 INFO :: lgZgr iterate: -8.878122022684071
2020-10-15 06:11:03,026 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.5e+00/s
2020-10-15 06:11:03,215 __main__ 0/1 INFO :: Perturbation norm: 0.004485446712911082
2020-10-15 06:11:03,216 __main__ 0/1 INFO :: lgT 

2020-10-15 06:11:36,662 __main__ 0/1 INFO :: L iterate: 0.008
-471.0568564937796
2020-10-15 06:11:36,883 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 7.8e+01/s
2020-10-15 06:11:36,895 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.5e+02/s
2020-10-15 06:11:36,898 __main__ 0/1 INFO :: Perturbation norm: 17.365219865144315
2020-10-15 06:11:36,899 __main__ 0/1 INFO :: T iterate: 2.268591126035416
2020-10-15 06:11:36,899 __main__ 0/1 INFO :: P iterate: 11.937870498209637
2020-10-15 06:11:36,934 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.2e+02/s
2020-10-15 06:11:36,938 __main__ 0/1 INFO :: Perturbation norm: 4.548874464065178
2020-10-15 06:11:36,938 __main__ 0/1 INFO :: T iterate: 2.4780799537499414
2020-10-15 06:11:36,939 __main__ 0/1 INFO :: P iterate: 12.686044882904369
2020-10-15 06:11:36,954 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s

2020-10-15 06:12:08,650 __main__ 0/1 INFO :: Perturbation norm: 15.323601797345436
2020-10-15 06:12:08,651 __main__ 0/1 INFO :: lgT iterate: 0.8938201385755977
2020-10-15 06:12:08,652 __main__ 0/1 INFO :: lgP iterate: 17.519430771656655
2020-10-15 06:12:08,653 __main__ 0/1 INFO :: lgM iterate: 15.604388704285158
2020-10-15 06:12:08,654 __main__ 0/1 INFO :: lgZgr iterate: -13.37556901393806
2020-10-15 06:12:10,894 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 4.0e+00/s
2020-10-15 06:12:11,075 __main__ 0/1 INFO :: Perturbation norm: 16.322203057556873
2020-10-15 06:12:11,076 __main__ 0/1 INFO :: lgT iterate: 0.8938201380224472
2020-10-15 06:12:11,077 __main__ 0/1 INFO :: lgP iterate: 17.519430765790272
2020-10-15 06:12:11,078 __main__ 0/1 INFO :: lgM iterate: 19.631103842564194
2020-10-15 06:12:11,079 __main__ 0/1 INFO :: lgZgr iterate: -16.06004576759541
2020-10-15 06:12:13,368 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, R

2020-10-15 06:12:48,040 __main__ 0/1 INFO :: lgZgr iterate: -20.27672596723233
2020-10-15 06:12:48,041 __main__ 0/1 INFO :: m iterate: 0.00012194126457565346
2020-10-15 06:12:48,042 __main__ 0/1 INFO :: L iterate: 0.039999999999999994
2020-10-15 06:12:49,988 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.0e+00/s
2020-10-15 06:12:50,929 __main__ 0/1 INFO :: Perturbation norm: 1.5386761195565955e-07
2020-10-15 06:12:50,930 __main__ 0/1 INFO :: lgT iterate: 0.8614210967325933
2020-10-15 06:12:50,931 __main__ 0/1 INFO :: lgP iterate: 18.393136560446795
2020-10-15 06:12:50,932 __main__ 0/1 INFO :: lgM iterate: 24.50335289384174
2020-10-15 06:12:50,933 __main__ 0/1 INFO :: lgZgr iterate: -20.276725970538074
2020-10-15 06:12:50,934 __main__ 0/1 INFO :: m iterate: 0.00012194117912045499
2020-10-15 06:12:50,934 __main__ 0/1 INFO :: L iterate: 0.04
-471.0568564937796
2020-10-15 06:12:51,150 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 

2020-10-15 06:13:15,829 __main__ 0/1 INFO :: lgM iterate: 13.401345534143402
2020-10-15 06:13:15,832 __main__ 0/1 INFO :: lgZgr iterate: -11.542473364096619
2020-10-15 06:13:18,231 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.9e+00/s
2020-10-15 06:13:18,400 __main__ 0/1 INFO :: Perturbation norm: 9.036743175488173
2020-10-15 06:13:18,400 __main__ 0/1 INFO :: lgT iterate: 0.8889842934766483
2020-10-15 06:13:18,404 __main__ 0/1 INFO :: lgP iterate: 17.147951370609697
2020-10-15 06:13:18,404 __main__ 0/1 INFO :: lgM iterate: 14.446363783698125
2020-10-15 06:13:18,408 __main__ 0/1 INFO :: lgZgr iterate: -12.239326766764005
2020-10-15 06:13:21,566 pencil 0/1 INFO :: Building pencil matrix 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 2.6e+00/s
2020-10-15 06:13:22,613 __main__ 0/1 INFO :: Perturbation norm: 9.370601959564755
2020-10-15 06:13:22,614 __main__ 0/1 INFO :: lgT iterate: 0.8889842933172742
2020-10-15 06:13:22,616 __main__ 0/1 INFO :: lg